In [ ]:
import os
from pathlib import Path

try:
    import google.colab

    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    from google.colab import userdata

    repo_name = "dgcnz/dl2"
    url = f"https://{userdata.get('gh_path')}@github.com/{repo_name}.git"
    !git clone {url}
    print("\nCurrent Directory:")
    %cd dl2
    #!pip install torch torchvision numpy matplotlib git+https://github.com/AMLab-Amsterdam/lie_learn escnn scipy
    !pip install torchvision git+https://github.com/AMLab-Amsterdam/lie_learn escnn lightning wandb git+https://github.com/dgcnz/neuralyze git+https://github.com/QUVA-Lab/e2cnn
    !pip install -r requirements.txt


else:  # automatically checks if the current directory is 'repo name'
    curdir = Path.cwd()
    print("Current Directory", curdir)
    assert (
        curdir.name == "dl2" or curdir.parent.name == "dl2"
    ), "Notebook cwd has to be on the project root"
    if curdir.name == "notebooks":
        %cd ..

In [ ]:
!git pull

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import gaussian_kde
import pandas as pd

In [ ]:
import wandb
wandb.login(key=userdata.get('wandb_key'))

In [ ]:
#path_dict = {'best_alpha_0.01_equivariance_level_9': 'uva-dl2/wang2022/model-ciun18ho:v49'}

path_dict_epoch = {'epoch_3_alpha_inf_equivariance_level_0': 'uva-dl2/wang2022/model-umb37s34:v3',
                   'epoch_47_alpha_inf_equivariance_level_0': 'uva-dl2/wang2022/model-umb37s34:v47',
                   'epoch_3_alpha_0_equivariance_level_0': 'uva-dl2/wang2022/model-8bep9a0w:v3',
                   'epoch_50_alpha_0_equivariance_level_0': 'uva-dl2/wang2022/model-8bep9a0w:v50'}

path_dict_best = {'epoch_11_alpha_inf_equivariance_level_9': 'uva-dl2/wang2022/model-4gmxxq83:v43',
                  'epoch_16_alpha_inf_equivariance_level_5': 'uva-dl2/wang2022/model-3wuzf1rf:v48',
                  'epoch_28_alpha_inf_equivariance_level_0': 'uva-dl2/wang2022/model-umb37s34:v47',
                  'epoch_12_alpha_0_equivariance_level_9': 'uva-dl2/wang2022/model-lgxaqwzp:v12',
                  'epoch_50_alpha_0.1_equivariance_level_9': 'uva-dl2/wang2022/model-kr959h28:v50',
                  'epoch_50_alpha_1e-6_equivariance_level_9': 'uva-dl2/wang2022/model-slhpjzjv:v50',
                  'epoch_49_alpha_0_equivariance_level_5':  'uva-dl2/wang2022/model-dkdkoxoe:v49',
                  'epoch_50_alpha_0.1_equivariance_level_5': 'uva-dl2/wang2022/model-re1ys3x2:v50',
                  'epoch_50_alpha_1e-6_equivariance_level_5': 'uva-dl2/wang2022/model-vp1v9jqj:v50',
                  'epoch_50_alpha_0_equivariance_level_0': 'uva-dl2/wang2022/model-8bep9a0w:v50',
                  'epoch_50_alpha_0.1_equivariance_level_0': 'uva-dl2/wang2022/model-fyw9jx76:v50',
                  'epoch_50_alpha_1e-6_equivariance_level_0': 'uva-dl2/wang2022/model-vz626xf7:v50'}


test_rmse_dict = {'epoch_11_alpha_inf_equivariance_level_9': 0.6712,
                  'epoch_16_alpha_inf_equivariance_level_5': 0.5139,
                  'epoch_28_alpha_inf_equivariance_level_0': 0.1022,
                  'epoch_12_alpha_0_equivariance_level_9': 0.08028,
                  'epoch_50_alpha_0.1_equivariance_level_9': 0.1115,
                  'epoch_50_alpha_1e-6_equivariance_level_9': 0.08612,
                  'epoch_49_alpha_0_equivariance_level_5':  0.4223,
                  'epoch_50_alpha_0.1_equivariance_level_5': 0.4209,
                  'epoch_50_alpha_1e-6_equivariance_level_5': 0.4188,
                  'epoch_50_alpha_0_equivariance_level_0': 0.638,
                  'epoch_50_alpha_0.1_equivariance_level_0': 0.6274 ,
                  'epoch_50_alpha_1e-6_equivariance_level_0': 0.6353}

dummy_dict = {'epoch_28_alpha_inf_equivariance_level_0': 'uva-dl2/wang2022/model-umb37s34:v47',
               'epoch_50_alpha_0_equivariance_level_0': 'uva-dl2/wang2022/model-8bep9a0w:v50'}


In [ ]:
!python -m src.compute_measures --help

In [ ]:
!git pull

In [ ]:
!python -m src.compute_measures ++weight_decay=1e-5 percentage_data=20 'ckpt_path_dict={path_dict_best}' spectrum=True sharpness=True

In [ ]:
import torch
tensor = torch.tensor(5.)
print(tensor.shape)

In [ ]:
with open('logs/measures/metrics (2).json') as json_file:
    metric_dict = json.load(json_file)
print(metric_dict)

##Hessian Spectra

In [ ]:
def plot_fn(spectrums: list, names: list, epoch: int):
    fig, ax = plt.subplots()
    # Create kernel density estimate
    for spectrum, name in zip(spectrums, names):
        kde = gaussian_kde(spectrum)

        # Create x values for the plot
        x = np.linspace(min(spectrum), max(spectrum), 1000)

        # Compute the density estimate
        density = kde(x)

        # Plot density estimate as a line
        ax.plot(x, density, label=name)

    # Add labels and title
    ax.set_xlabel("Max Eigenvalue")
    ax.set_ylabel("Density")
    ax.set_title("Density plot of max eigenvalues at epoch " + str(epoch))
    ax.legend()
    plt.show()

In [ ]:
names = [name for name in metric_dict.keys()]
spectrums = [values['spectrum'] for values in metric_dict.values()]
plot_fn(spectrums, names, 'best')

##Sharpeness


##Equivariance learned/error

In [ ]:
equivariance_dict = {'alpha': [], 'equivariance_level': [], 'error': [], 'lie': [], 'sharpness':[]}
data_equivariance_error_map = {0: 0.0,1: 0.17,2: 0.313,3: 0.435,4: 0.541,5: 0.63,6: 0.709,7: 0.78,
                               8:0.84, 9:0.896}
#epoch_11_alpha_inf_equivariance_level_9

for key, value in metric_dict.items():
    error = value['equivariance_error']
    lie = value['lie_derivative']
    sharpness = value['sharpness']

    parts = key.split('_')
    alpha = parts[3]
    alpha = float(alpha)
    level = parts[-1]
    equivariance_dict['alpha'].append(alpha)
    equivariance_dict['equivariance_level'].append(int(level))
    equivariance_dict['error'].append(error)
    equivariance_dict['lie'].append(lie)
    equivariance_dict['sharpness'].append(sharpness)

df = pd.DataFrame(equivariance_dict)

print(df)

In [ ]:
##Error

# set width of bar
barWidth = 0.25
fig = plt.subplots(figsize =(12, 8))

# set height of bar
level_0 = df[df['equivariance_level'] == 0]['error']
level_5 = df[df['equivariance_level'] == 5]['error']
level_9 = df[df['equivariance_level'] == 9]['error']

# Set position of bar on X axis
br1 = np.arange(len(level_0))
br2 = [x + barWidth for x in br1]
br3 = [x + barWidth for x in br2]

# Create bars with adjusted colors
plt.bar(br1, level_0, color='#F9D342', width=barWidth, edgecolor='grey', label='Data Equivariance Error ' + str(data_equivariance_error_map[0]))
plt.bar(br2, level_5, color='#FF8C42', width=barWidth, edgecolor='grey', label='Data Equivariance Error ' + str(data_equivariance_error_map[5]))
plt.bar(br3, level_9, color='#FF6A6A', width=barWidth, edgecolor='grey', label='Data Equivariance Error ' + str(data_equivariance_error_map[9]))

# Adding Xticks
plt.xlabel('Alpha', fontsize=15)
plt.ylabel('Model Equivariance Error', fontsize=15)
plt.xticks([r + barWidth for r in range(len(level_0))], ['0', '1e-6', '1e-2', 'Equivariant Net'])

# Adding grid
plt.grid(True, which='major', linestyle='--', linewidth='0.5', color='grey')

# Add legend
plt.legend()

# Display the plot
plt.show()

In [ ]:
#Lie

# set width of bar
barWidth = 0.25
fig = plt.subplots(figsize =(12, 8))

# set height of bar
level_0 = df[df['equivariance_level'] == 0]['lie']
level_5 = df[df['equivariance_level'] == 5]['lie']
level_9 = df[df['equivariance_level'] == 9]['lie']

# Set position of bar on X axis
br1 = np.arange(len(level_0))
br2 = [x + barWidth for x in br1]
br3 = [x + barWidth for x in br2]

# Create bars with adjusted colors
plt.bar(br1, level_0, color='#F9D342', width=barWidth, edgecolor='grey', label='Data Equivariance Error ' + str(data_equivariance_error_map[0]))
plt.bar(br2, level_5, color='#FF8C42', width=barWidth, edgecolor='grey', label='Data Equivariance Error ' + str(data_equivariance_error_map[5]))
plt.bar(br3, level_9, color='#FF6A6A', width=barWidth, edgecolor='grey', label='Data Equivariance Error ' + str(data_equivariance_error_map[9]))

# Adding Xticks
plt.xlabel('Alpha', fontsize=15)
plt.ylabel('Model Lie Derivative', fontsize=15)
plt.xticks([r + barWidth for r in range(len(level_0))], ['0', '1e-6', '1e-2', 'Equivariant Net'])

# Adding grid
plt.grid(True, which='major', linestyle='--', linewidth='0.5', color='grey')

# Add legend
plt.legend()

# Display the plot
plt.show()

In [ ]:
#Lie

# set width of bar
barWidth = 0.25
fig = plt.subplots(figsize =(12, 8))


# set height of bar
level_0 = df[df['equivariance_level'] == 0]['sharpness']
level_5 = df[df['equivariance_level'] == 5]['sharpness']
level_9 = df[df['equivariance_level'] == 9]['sharpness']

# Set position of bar on X axis
br1 = np.arange(len(level_0))
br2 = [x + barWidth for x in br1]
br3 = [x + barWidth for x in br2]

# Create bars with adjusted colors
plt.bar(br1, level_0, color='#F9D342', width=barWidth, edgecolor='grey', label='Data Equivariance Error ' + str(data_equivariance_error_map[0]))
plt.bar(br2, level_5, color='#FF8C42', width=barWidth, edgecolor='grey', label='Data Equivariance Error ' + str(data_equivariance_error_map[5]))
plt.bar(br3, level_9, color='#FF6A6A', width=barWidth, edgecolor='grey', label='Data Equivariance Error ' + str(data_equivariance_error_map[9]))

# Adding Xticks
plt.xlabel('Alpha', fontsize=15)
plt.ylabel('Model Lie Derivative', fontsize=15)
plt.xticks([r + barWidth for r in range(len(level_0))], ['0', '1e-6', '1e-2', 'Equivariant Net'])

# Adding grid
plt.grid(True, which='major', linestyle='--', linewidth='0.5', color='grey')

# Add legend
plt.legend()

# Display the plot
plt.show()

In [ ]:
#Lie
equivariance_dict = {'epoch': [], 'sharpness':[], 'alpha':[]}

#epoch_11_alpha_inf_equivariance_level_9

for key, value in metric_dict.items():
    sharpness = value['sharpness']


    parts = key.split('_')
    epoch = parts[1]
    alpha = parts[3]

    equivariance_dict['epoch'].append(epoch)
    equivariance_dict['sharpness'].append(sharpness)
    equivariance_dict['alpha'].append(alpha)

# set width of bar
df = pd.DataFrame(equivariance_dict)
df = df.sort_values(by='epoch')

print(df)
print('hello')

barWidth = 0.25
fig = plt.subplots(figsize =(12, 8))

# set height of bar
E2 = df[df['alpha'] == 'inf']['sharpness']
Rsteer = df[df['alpha'] == '0']['sharpness']


# Set position of bar on X axis
br1 = np.arange(len(E2))
br2 = [x + barWidth for x in br1]

# Create bars with adjusted colors
plt.bar(br1, E2, color='#4fa89e', width=barWidth, edgecolor='grey', label='E2CNN')
plt.bar(br2, Rsteer, color='#eda7be', width=barWidth, edgecolor='grey', label='Rsteer')

# Adding Xticks
plt.xlabel('Checkpoint', fontsize=15)
plt.ylabel('Sharpness', fontsize=15)
plt.xticks([r + barWidth / 2 for r in range(len(E2))], ['Early', 'Best'])
plt.yscale('log')
# Adding grid
plt.grid(True, which='major', linestyle='--', linewidth='0.5', color='grey')

# Add legend
plt.legend()

# Display the plot
plt.show()

###RMSE###


In [ ]:
df_dict = {'alpha': [], 'equivariance_level': [], 'value': []}
data_equivariance_error_map = {0: 0.0,1: 0.17,2: 0.313,3: 0.435,4: 0.541,5: 0.63,6: 0.709,7: 0.78,
                               8:0.84, 9:0.896}
#epoch_11_alpha_inf_equivariance_level_9
print(test_rmse_dict)

for key, value in test_rmse_dict.items():
    parts = key.split('_')
    alpha = parts[3]
    alpha = float(alpha)
    level = parts[-1]
    df_dict['alpha'].append(alpha)
    df_dict['equivariance_level'].append(
        data_equivariance_error_map[int(level)])
    df_dict['value'].append(value)

# Creating the DataFrame
df = pd.DataFrame(df_dict)

# Converting alpha and equivariance_level to numeric types for plotting
df['alpha'] = pd.to_numeric(df['alpha'])
df['equivariance_level'] = pd.to_numeric(df['equivariance_level'])

# Sorting the DataFrame by the alpha column
df = df.sort_values(by='alpha')

print(df)

#################### MAKE the bar plot #####################
# set width of bar
barWidth = 0.25
fig = plt.subplots(figsize =(12, 8))

# set height of bar
#IT = [12, 30, 1, 8, 22]
level_0 = df[df['equivariance_level'] == data_equivariance_error_map[0]]['rmse']


# Set position of bar on X axis
br1 = np.arange(len(level_0))
br2 = [x + barWidth for x in br1]
br3 = [x + barWidth for x in br2]

# Make the plot
plt.bar(br1, level_0, color ='r', width = barWidth,
        edgecolor ='grey', label ='IT')
#plt.bar(br2, ECE, color ='g', width = barWidth,
#       edgecolor ='grey', label ='ECE')
#lt.bar(br3, CSE, color ='b', width = barWidth,
#        edgecolor ='grey', label ='CSE')

# Adding Xticks
plt.xlabel('Branch', fontweight ='bold', fontsize = 15)
plt.ylabel('Students passed', fontweight ='bold', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(level_0))],
        ['0', '0.1', '0.000001'])

plt.legend()
plt.show()







In [ ]:
# @title alpha vs value

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(df['alpha'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(df, x='value', y='alpha', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)